In [1]:
ls

0_To generate n-best hypothesis.ipynb    torgo.csv
1_To generate audio_features.ipynb       torgo_test_single.ipynb
2_finetune_whisper_on_torgo-Copy1.ipynb  Untitled.ipynb
3_prepare_for_torgo_baseline.ipynb       whisper-tiny/
output/


In [1]:
pwd

'/work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA/data_preparation'

In [5]:
!sbatch tinf.bash

Submitted batch job 42938168


In [2]:
!sbatch ttrain.bash

Submitted batch job 42896850


In [8]:
!sbatch tdata.bash

Submitted batch job 42795999


In [2]:
!squeue -u zhang.jinda1

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          42950955       gpu tinf.bas zhang.ji  R      29:38      1 d1007
          42950999     short sys/dash zhang.ji  R      24:35      1 c0177


In [3]:
!scancel 42935852

In [2]:
import json
with open('training_args.json') as training_args_file:
    training_args_dict = json.load(training_args_file)

# Replace the default values with the values from the command line arguments
training_args_dict['learning_rate'] = learning_rate
training_args_dict['per_device_train_batch_size'] = train_batch_size
training_args_dict['per_device_eval_batch_size'] = eval_batch_size
training_args_dict['seed'] = seed
training_args_dict['gradient_accumulation_steps'] = gradient_accumulation_steps
training_args_dict['optim'] = optimizer
training_args_dict['lr_scheduler_type'] = lr_scheduler_type
training_args_dict['num_train_epochs'] = num_epochs

training_args = TrainingArguments(
    output_dir=model_local_path,
    hub_model_id=repo_name,
    **training_args_dict
)

NameError: name 'learning_rate' is not defined

In [6]:
import os
import glob

def delete_files_with_prefix(prefix):
    # Get the current working directory
    current_dir = os.getcwd()

    # Create a search pattern for files starting with the given prefix
    search_pattern = os.path.join(current_dir, f"{prefix}*")

    # Find all files matching the search pattern
    files_to_delete = glob.glob(search_pattern)

    # Iterate over the files and delete them
    for file_path in files_to_delete:
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

# Example usage
prefix = "slurm-4271395"  # Replace with your actual prefix
delete_files_with_prefix(prefix)

Deleted: /work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA/data_preparation/slurm-42713952.out


In [5]:
import json
with open('training_args.json') as training_args_file:
    training_args_dict = json.load(training_args_file)

In [6]:
training_args_dict

{'evaluation_strategy': 'steps',
 'per_device_train_batch_size': 4,
 'per_device_eval_batch_size': 4,
 'gradient_accumulation_steps': 2,
 'eval_delay': 0,
 'learning_rate': 0.0001,
 'weight_decay': 0.005,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'max_steps': -1,
 'lr_scheduler_type': 'linear',
 'warmup_ratio': 0.0,
 'warmup_steps': 1000,
 'save_strategy': 'steps',
 'save_steps': 500,
 'save_total_limit': 3,
 'report_to': 'all',
 'seed': 42,
 'eval_steps': 1000,
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'optim_args': None,
 'adafactor': False,
 'group_by_length': True,
 'length_column_name': 'length',
 'push_to_hub': True,
 'hub_strategy': 'every_save'}

In [1]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load the tokenizer, retriever, and model
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", use_dummy_dataset=True)
model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)

# Define the input query
input_query = "What is the capital of France?"

# Tokenize the input query
input_ids = tokenizer(input_query, return_tensors="pt").input_ids

# Generate the response
output_ids = model.generate(input_ids)

# Decode the generated response
response = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

print("Query:", input_query)
print("Response:", response)

/work/van-speech-nlp/jindaznb/mmenv/lib/python3.9/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


ImportError: 
RagRetriever requires the faiss library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/facebookresearch/faiss/blob/master/INSTALL.md and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [10]:
import torch
!cd ..
data_path = "Inference/gs_inferences/torgo_F01_test.pt"
data   = torch.load(data_path,map_location=torch.device('cpu'))

In [9]:
cd ..

/work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA


In [11]:
len(data)

460

In [12]:
import torch
!cd ..
data_path = "Inference/gs_inferences/torgo_F01_val.pt"
data   = torch.load(data_path,map_location=torch.device('cpu'))

len(data)

460

In [ ]:
!speaker_id='F01' && python training/WL-S-Copy1.py --lr 1e-3 \
    --d 1 \
    --pretrained_path 'weights/alpaca.pth' \
    --tokenizer_path 'weights/tokenizer.model' \
    --data 'Inference/gs_inferences/torgo_${speaker_id}'

In [3]:
from datasets import load_dataset
from pprint import pprint

# Load the HyPoradise-v1-GigaSpeech dataset
dataset = load_dataset("PeacefulData/HyPoradise-v1-GigaSpeech")

# Example: Print the first example from the train split
pprint(dataset['train'][0])

{'ID': 'YOU0000000145_S0000030',
 'Unnamed: 0': 0,
 'audio_time': 1.67999267578125,
 'category': 25,
 'category_name': 'Entertainment',
 'ground_truth': 'so close to my mouth.',
 'hypothesis_1': 'its so close to my mouth',
 'hypothesis_10': 'socialist tomorrow',
 'hypothesis_11': 'suitable so close to my mouth',
 'hypothesis_12': 'sılcostzamerio',
 'hypothesis_13': 'thats so close to my mouth',
 'hypothesis_14': None,
 'hypothesis_15': None,
 'hypothesis_16': None,
 'hypothesis_17': None,
 'hypothesis_18': None,
 'hypothesis_19': None,
 'hypothesis_2': 'seriously sewing',
 'hypothesis_20': None,
 'hypothesis_3': 'so close to mama',
 'hypothesis_4': 'so close to my mall',
 'hypothesis_5': 'so close to my mouth',
 'hypothesis_6': 'so close to my mouth butt',
 'hypothesis_7': 'so close to my mouth second up',
 'hypothesis_8': 'so close to my mouth!',
 'hypothesis_9': 'so close to your mouth',
 'source': 2}


In [2]:
len(dataset['train'])

32739

In [4]:
len(dataset['test'])

KeyError: 'test'

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'category_name', 'ground_truth', 'hypothesis_1', 'hypothesis_2', 'hypothesis_3', 'hypothesis_4', 'hypothesis_5', 'hypothesis_6', 'hypothesis_7', 'hypothesis_8', 'hypothesis_9', 'hypothesis_10', 'hypothesis_11', 'hypothesis_12', 'hypothesis_13', 'hypothesis_14', 'hypothesis_15', 'hypothesis_16', 'hypothesis_17', 'hypothesis_18', 'hypothesis_19', 'hypothesis_20', 'source', 'category', 'audio_time', 'ID'],
        num_rows: 32739
    })
})

In [14]:
import base64
import numpy as np

_ALIGNMENT_HEADS = {
    "tiny.en": b"ABzY8J1N>@0{>%R00Bk>$p{7v037`oCl~+#00",
    # Add other models as needed...
}

def get_alignment_heads(model_key):
    if model_key not in _ALIGNMENT_HEADS:
        raise ValueError("Model key not found in alignment heads dictionary")
    
    encoded_string = _ALIGNMENT_HEADS[model_key]
    # Decode the base85 string
    decoded_bytes = base64.b85decode(encoded_string)
    # Convert bytes to boolean array
    bool_array = np.unpackbits(np.frombuffer(decoded_bytes, dtype=np.uint8)).astype(bool)
    
    return bool_array

# Example usage
model_key = "tiny.en"
alignment_heads = get_alignment_heads(model_key)
print(alignment_heads)

[False False False  True  True  True  True  True  True False False False
  True False  True  True False False False False  True False False False
 False False False False False False False False False False  True  True
  True False  True  True False False  True False  True False False  True
  True  True False  True  True False  True False False  True  True False
 False False  True  True False False False False False False  True False
  True  True  True  True  True  True  True  True False  True  True False
 False False  True  True False  True  True False False False False False
 False False False False False False False False False False False False
 False False False  True False  True False False False  True  True False
 False False False False False  True False False  True  True False False
  True False False  True False False False False  True False False False
 False False False False False  True  True False False False False False
  True  True False False False False False False Fa

In [28]:
a = "Hello, "
b = None

result = a + b  # This will raise a TypeError

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
result

In [11]:
import whisper


KeyboardInterrupt



In [16]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 5.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=995b9db7711ce59e938a8c81e0fd0de2acb56fb588accc01dcfeb1a1ffdc6999
  Stored in directory: /home/zhang.jinda1/.cache/pip/wheels/f5/77/96/4bb7b94449a47b726127100ad66bd72cba123fb4d0a8948473
Successfully built openai-whisper


In [17]:
import whisper

In [ ]:
model = whisper.load_model("large-v3")

In [ ]:
model

In [8]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import soundfile as sf
import torch

# Load pretrained model and tokenizer
tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53")

# The model was trained on 16 kHz sampled speech audio
# If your speech audio is not 16 kHz, you might need to resample it
speech, _ = sf.read("path_to_your_audio_file.wav")

# Tokenize the speech
input_values = tokenizer(speech, return_tensors="pt").input_values

# Forward pass through the model
logits = model(input_values).logits

# Take argmax to get the predicted token ids
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the token ids to get the transcription
transcription = tokenizer.decode(predicted_ids[0])

print("Transcription: ", transcription)

/work/van-speech-nlp/jindaznb/mmenv/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:53: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: Can't load tokenizer for 'facebook/wav2vec2-large-xlsr-53'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'facebook/wav2vec2-large-xlsr-53' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [9]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio

# Load the pre-trained Wav2Vec2-XLSR-53 model and processor
model_name = "facebook/wav2vec2-large-xlsr-53"
model = Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OSError: Can't load tokenizer for 'facebook/wav2vec2-large-xlsr-53'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'facebook/wav2vec2-large-xlsr-53' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [1]:
# Load a Pre-Trained Checkpoint
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [2]:
model.freeze_feature_encoder()

AttributeError: 'WhisperForConditionalGeneration' object has no attribute 'freeze_feature_encoder'

In [4]:

# Freeze the feature encoder
for name, param in model.named_parameters():
    if 'encoder' in name:  # Assuming 'encoder' is in the name of the feature encoder layers
        param.requires_grad = False

# Confirm that the feature encoder has been frozen
for name, param in model.named_parameters():
    if 'encoder' in name:
        print(name, param.requires_grad)  # This should print False for all encoder parameters

model.encoder.conv1.weight False
model.encoder.conv1.bias False
model.encoder.conv2.weight False
model.encoder.conv2.bias False
model.encoder.embed_positions.weight False
model.encoder.layers.0.self_attn.k_proj.weight False
model.encoder.layers.0.self_attn.v_proj.weight False
model.encoder.layers.0.self_attn.v_proj.bias False
model.encoder.layers.0.self_attn.q_proj.weight False
model.encoder.layers.0.self_attn.q_proj.bias False
model.encoder.layers.0.self_attn.out_proj.weight False
model.encoder.layers.0.self_attn.out_proj.bias False
model.encoder.layers.0.self_attn_layer_norm.weight False
model.encoder.layers.0.self_attn_layer_norm.bias False
model.encoder.layers.0.fc1.weight False
model.encoder.layers.0.fc1.bias False
model.encoder.layers.0.fc2.weight False
model.encoder.layers.0.fc2.bias False
model.encoder.layers.0.final_layer_norm.weight False
model.encoder.layers.0.final_layer_norm.bias False
model.encoder.layers.1.self_attn.k_proj.weight False
model.encoder.layers.1.self_attn.v_

In [5]:
for name, param in model.named_parameters():

    print(name, param.requires_grad)  # This should print False for all encoder parameters

model.encoder.conv1.weight Parameter containing:
tensor([[[-7.5645e-03, -7.5073e-03, -7.2098e-03],
         [ 2.4460e-02,  1.7090e-02,  1.2566e-02],
         [ 1.7639e-02,  1.3382e-02,  7.1297e-03],
         ...,
         [-4.5654e-02, -2.0615e-02, -1.8799e-02],
         [-5.6152e-02, -3.0167e-02, -2.2278e-02],
         [-9.1309e-02, -7.4463e-02, -6.0150e-02]],

        [[ 1.9836e-03,  4.9531e-05,  8.9417e-03],
         [ 3.6182e-03, -4.6921e-03,  7.0953e-03],
         [ 1.3229e-02,  2.4738e-03,  1.4656e-02],
         ...,
         [ 1.0986e-02,  1.0757e-02, -6.6795e-03],
         [ 7.4883e-03,  4.8370e-03, -1.3214e-02],
         [ 2.4628e-02,  2.2186e-02,  2.5005e-03]],

        [[ 2.1042e-02,  2.7985e-02,  3.2715e-02],
         [-6.1737e-02, -7.9346e-02, -9.3628e-02],
         [-2.3315e-02, -5.3467e-02, -5.6549e-02],
         ...,
         [-9.7942e-04, -8.1940e-03, -2.5139e-03],
         [ 2.8801e-03, -4.2114e-03, -3.9139e-03],
         [-9.0561e-03, -1.6479e-02, -7.9575e-03]],

   